In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

video_path = 'video.mp4'

cap = cv2.VideoCapture(video_path)

sw = 5
sh = 3
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

sift = cv2.SIFT_create()
bf = cv2.BFMatcher(crossCheck=True)

final_image = None

ret, prev_frame0 = cap.read()
if not ret:
    print("Error: Could not read the first frame.")
    cap.release()
    exit()


h, w, _ = prev_frame0.shape
prev_frame = np.zeros((h*sh, w*sw, 3), dtype=np.uint8)
prev_frame[(sh//2)*h:(sh//2+1)*h, (sw//2)*w:(sw//2+1)*w, :] = prev_frame0

prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

prev_kp, prev_des = sift.detectAndCompute(prev_frame_gray, None)
cnt = 0
brdr = ((sh//2)*h, (sh//2+1)*h, (sw//2)*w, (sw//2+1)*w)
while True:
    cnt += 1
    ret, curr_frame = cap.read()
    if not ret:
        break

    ccf = np.zeros((h*sh, w*sw, 3), dtype=np.uint8)
    ccf[(sh//2)*h:(sh//2+1)*h, (sw//2)*w:(sw//2+1)*w, :] = curr_frame

    sift = cv2.SIFT_create()

    kp1, des1 = sift.detectAndCompute(prev_frame, None)
    kp2, des2 = sift.detectAndCompute(ccf, None)


    # BFMatcher with default params. Default norm is NORM_L2. Default crossCheck is False. CrossCheck=True means Matcher returns only those matches with value (i,j) such that i-th descriptor in set A has j-th descriptor in set B as the best match and vice-versa. That is, the two features in both sets should match each other. Good substitute for ratio test
    bf = cv2.BFMatcher(crossCheck=True)
    # Match descriptors.
    matches = bf.match(des1, des2)

    # Sort them in the order of their distance.
    sortmatches = sorted(matches, key = lambda x:x.distance)



    dst_pts = np.float32([kp1[m.queryIdx].pt for m in sortmatches[:50]]).reshape(-1, 1, 2)
    src_pts = np.float32([kp2[m.trainIdx].pt for m in sortmatches[:50]]).reshape(-1, 1, 2)

    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)

    if abs(np.linalg.det(H) - 1) < 0.01 and cnt > 1:
        continue

    imgWarp = cv2.warpPerspective(ccf, H, (w*sw, h*sh))

    ccm = np.zeros((h*sh, w*sw), dtype=np.uint8)
    ccm[(sh//2)*h:(sh//2+1)*h, (sw//2)*w:(sw//2+1)*w] = np.ones((h, w), dtype=np.uint8)
    curr_mask = cv2.warpPerspective(ccm, H, (w*sw, h*sh))

    # ایجاد ماسک‌های وزنی
    prev_mask = (prev_frame.sum(axis=2) > 0).astype(np.float32)

    combined_mask = curr_mask + prev_mask
    curr_weight = curr_mask / combined_mask
    final_weight = prev_mask / combined_mask

    for c in range(3):
        prev_frame[..., c] = (
            curr_weight * imgWarp[..., c] + final_weight * prev_frame[..., c]
        ).astype(np.uint8)


    # prev_frame = np.where(curr_mask == 1, imgWarp, prev_frame)

    plt.imshow(cv2.cvtColor(prev_frame, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
cap.release()

